In [ ]:
import pandas as pd
import numpy as np
import tensorflow as TF
from typing import Tuple, List
import cv2
import os
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt
CATEGORIES = [0, 1, 2, 3, 4, 5]
EPOCHS = 50
TEST = 0.3

In [69]:
def load_signs_dataset(dataDir : str): # type: ignore
    images = []
    labels = []
    for SingleCategory in CATEGORIES:
        category = os.path.join(dataDir, str(SingleCategory))
        
        for image in os.listdir(category):
            img_path = os.path.join(category, image)
            finalImage = cv2.imread(img_path)
            
            if finalImage is not None:
                img = cv2.resize(finalImage, (64, 64))
                images.append(img)
                
                labels.append(SingleCategory)
    
    return (np.array(images), np.array(labels))
                

In [70]:
images, labels = load_signs_dataset("DataSet/archive")

X_train_orig, X_test_orig, Y_train_orig, Y_test_orig= train_test_split(
    np.array(images), np.array(labels), test_size=TEST
)

print(X_train_orig.shape)
print(Y_train_orig.shape)
print(X_test_orig.shape)
print(Y_test_orig.shape)

(865, 64, 64, 3)
(865,)
(372, 64, 64, 3)
(372,)


In [71]:
def model(input_shape):
    input_img = TF.keras.Input(shape=input_shape)
    Z1 = TF.keras.layers.Conv2D(filters=8, kernel_size=(4,4), padding="same")(input_img)
    
    A1 = TF.keras.layers.ReLU()(Z1)
    
    P1 = TF.keras.layers.MaxPool2D(pool_size=(8,8), strides=8, padding="same")(A1)
    
    Z2 = TF.keras.layers.Conv2D(filters=16, kernel_size=(2,2), strides=1, padding="same")(P1)
    
    A2 = TF.keras.layers.ReLU()(Z2)
    
    P2 = TF.keras.layers.MaxPool2D(pool_size=(4,4), strides=4, padding="same")(A2)
    
    
    F = TF.keras.layers.Flatten()(P2)
    
    outpus = TF.keras.layers.Dense(units= 6, activation="softmax")(F)
    
    model = TF.keras.Model(input_img,outpus)
    return model

In [79]:
ConvModel = model((64,64,3))
ConvModel.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
    )

Y_train_oh = TF.keras.utils.to_categorical(Y_train_orig, num_classes=6)
Y_test_oh = TF.keras.utils.to_categorical(Y_test_orig, num_classes=6)
history = ConvModel.fit(X_train_orig/255.0, Y_train_oh, epochs=EPOCHS, validation_data=(X_test_orig/255.0, Y_test_oh))
ConvModel.summary()


Epoch 1/50
28/28 [==============================] - 3s 60ms/step - loss: 1.8163 - accuracy: 0.1630 - val_loss: 1.7992 - val_accuracy: 0.1694
Epoch 2/50
28/28 [==============================] - 1s 36ms/step - loss: 1.7890 - accuracy: 0.2058 - val_loss: 1.7907 - val_accuracy: 0.1962
Epoch 3/50
28/28 [==============================] - 1s 39ms/step - loss: 1.7817 - accuracy: 0.2301 - val_loss: 1.7897 - val_accuracy: 0.1317
Epoch 4/50
28/28 [==============================] - 1s 37ms/step - loss: 1.7769 - accuracy: 0.2324 - val_loss: 1.7836 - val_accuracy: 0.2446
Epoch 5/50
28/28 [==============================] - 1s 36ms/step - loss: 1.7724 - accuracy: 0.2382 - val_loss: 1.7731 - val_accuracy: 0.2500
Epoch 6/50
28/28 [==============================] - 1s 36ms/step - loss: 1.7633 - accuracy: 0.2613 - val_loss: 1.7633 - val_accuracy: 0.2285
Epoch 7/50
28/28 [==============================] - 1s 37ms/step - loss: 1.7573 - accuracy: 0.2173 - val_loss: 1.7650 - val_accuracy: 0.1452
Epoch 8/50
28

In [80]:
test_loss, img_accuracy = ConvModel.evaluate(X_test_orig, Y_test_oh)

if img_accuracy >= 0.85:
    ConvModel.save("HandDetector.h5")

12/12 [==============================] - 0s 11ms/step - loss: 56.0593 - accuracy: 0.8226


In [81]:
df_loss_acc = pd.DataFrame(history.history)
df_loss= df_loss_acc[['loss','val_loss']]
df_loss.rename(columns={'loss':'train','val_loss':'validation'},inplace=True)
df_acc= df_loss_acc[['accuracy','val_accuracy']]
df_acc.rename(columns={'accuracy':'train','val_accuracy':'validation'},inplace=True)
df_loss.plot(title='Model loss',figsize=(12,8)).set(xlabel='Epoch',ylabel='Loss')
df_acc.plot(title='Model Accuracy',figsize=(12,8)).set(xlabel='Epoch',ylabel='Accuracy')

C:\Users\wedell\AppData\Local\Temp\ipykernel_16724\974158325.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_loss.rename(columns={'loss':'train','val_loss':'validation'},inplace=True)
C:\Users\wedell\AppData\Local\Temp\ipykernel_16724\974158325.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_acc.rename(columns={'accuracy':'train','val_accuracy':'validation'},inplace=True)


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.